In [17]:
import numpy as np
import os
import train_dense
import train_cnn

In [18]:
def load_mnist():
    data_dir = '../data'

    fd = open(os.path.join(data_dir, 'train-images-idx3-ubyte'))
    loaded = np.fromfile(file=fd, dtype=np.uint8)
    trX = loaded[16:].reshape((60000, 28, 28, 1)).astype(np.float)

    fd = open(os.path.join(data_dir, 'train-labels-idx1-ubyte'))
    loaded = np.fromfile(file=fd, dtype=np.uint8)
    trY = loaded[8:].reshape((60000)).astype(np.int)

    fd = open(os.path.join(data_dir, 't10k-images-idx3-ubyte'))
    loaded = np.fromfile(file=fd, dtype=np.uint8)
    teX = loaded[16:].reshape((10000, 28, 28, 1)).astype(np.float)

    fd = open(os.path.join(data_dir, 't10k-labels-idx1-ubyte'))
    loaded = np.fromfile(file=fd, dtype=np.uint8)
    teY = loaded[8:].reshape((10000)).astype(np.int)

    trY = np.asarray(trY)
    teY = np.asarray(teY)

    perm = np.random.permutation(trY.shape[0])
    trX = trX[perm]
    trY = trY[perm]

    perm = np.random.permutation(teY.shape[0])
    teX = teX[perm]
    teY = teY[perm]

    return trX, trY, teX, teY

In [19]:
trainX, trainY, testX, testY = load_mnist()

In [20]:
# Imports
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import tensorflow as tf

from tensorflow.contrib import learn
from tensorflow.contrib.learn.python.learn.estimators import model_fn as model_fn_lib

tf.logging.set_verbosity(tf.logging.INFO)

In [21]:
def cnn_model_fn(features, labels, mode):
    """Model function for CNN."""
    # Input Layer
    input_layer = tf.reshape(features, [-1, 28, 28, 1])

    # Convolutional Layer #1
    conv1 = tf.layers.conv2d(
      inputs=input_layer,
      filters=32,
      kernel_size=[5, 5],
      padding="same",
      activation=tf.nn.relu)

    # Pooling Layer #1
    pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2)

    # Convolutional Layer #2 and Pooling Layer #2
    conv2 = tf.layers.conv2d(
      inputs=pool1,
      filters=64,
      kernel_size=[5, 5],
      padding="same",
      activation=tf.nn.relu)
    pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2)

    # Dense Layer
    pool2_flat = tf.reshape(pool2, [-1, 7 * 7 * 64])
    dense = tf.layers.dense(inputs=pool2_flat, units=1024, activation=tf.nn.relu)
    dropout = tf.layers.dropout(
      inputs=dense, rate=0.4, training=mode == learn.ModeKeys.TRAIN)

    # Logits Layer
    logits = tf.layers.dense(inputs=dropout, units=10)

    loss = None
    train_op = None

    # Calculate Loss (for both TRAIN and EVAL modes)
    if mode != learn.ModeKeys.INFER:
        onehot_labels = tf.one_hot(indices=tf.cast(labels, tf.int32), depth=10)
        loss = tf.losses.softmax_cross_entropy(
            onehot_labels=onehot_labels, logits=logits)

    # Configure the Training Op (for TRAIN mode)
    if mode == learn.ModeKeys.TRAIN:
        train_op = tf.contrib.layers.optimize_loss(
            loss=loss,
            global_step=tf.contrib.framework.get_global_step(),
            learning_rate=0.001,
            optimizer="SGD")

    # Generate Predictions
    predictions = {
      "classes": tf.argmax(
          input=logits, axis=1),
      "probabilities": tf.nn.softmax(
          logits, name="softmax_tensor")
    }

    # Return a ModelFnOps object
    return model_fn_lib.ModelFnOps(
      mode=mode, predictions=predictions, loss=loss, train_op=train_op)

In [26]:
# def main(unused_argv):
    # Load training and eval data
# mnist = learn.datasets.load_dataset("mnist")
# trainX, trainY, testX, testY = load_mnist()
train_data = np.asarray(trainX, dtype=np.float32) # Returns np.array
train_labels = np.asarray(trainY, dtype = np.float32)
eval_data = np.asarray(testX, dtype = np.float32) # Returns np.array
eval_labels = np.asarray(testY, dtype = np.float32)

# Estimator
mnist_classifier = learn.Estimator(
  model_fn=cnn_model_fn, model_dir="tmp2")

# Logging hook
tensors_to_log = {"probabilities": "softmax_tensor"}
logging_hook = tf.train.LoggingTensorHook(
  tensors=tensors_to_log, every_n_iter=50)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_keep_checkpoint_every_n_hours': 10000, '_environment': 'local', '_master': '', '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1
}
, '_task_id': 0, '_tf_random_seed': None, '_task_type': None, '_save_checkpoints_secs': 600, '_evaluation_master': '', '_save_summary_steps': 100, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x0000018DE6A5BDD8>, '_keep_checkpoint_max': 5, '_num_ps_replicas': 0, '_save_checkpoints_steps': None, '_is_chief': True}


In [6]:
trainX.shape

(60000, 28, 28, 1)

In [24]:
mnist_classifier.fit(
    x=train_data,
    y=train_labels,
    batch_size=100,
    steps=2000,
    monitors=[logging_hook])

Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Example conversion:
  est = Estimator(...) -> est = SKCompat(Estimator(...))
Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Example conversion:
  est = Estimator(...) -> est = SKCompat(Estimator(...))
Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Example conversion:
  est = Estimator(...) -> est = SKCompat(Estimator(...))


D:\Program Files\Anaconda3\envs\tensorflow_tuts\lib\site-packages\tensorflow\python\util\deprecation.py:247: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.
  equality = a == b


INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into tmp/mnist_convnet_model\model.ckpt.
INFO:tensorflow:loss = 44.5948, step = 1
INFO:tensorflow:probabilities = [[  1.06465947e-07   1.79288461e-18   2.75034145e-05   1.84162423e-01
    6.50300170e-12   3.93921606e-09   1.96547920e-20   8.15799057e-01
    1.13718726e-16   1.08909790e-05]
 [  1.00000000e+00   3.99342877e-39   7.73115364e-14   5.37979210e-11
    8.34940597e-34   2.47404907e-17   2.49373773e-25   5.22350607e-09
    7.47729145e-24   5.23765564e-08]
 [  1.22537361e-02   1.79641054e-26   8.51732044e-21   9.87271726e-01
    6.29713211e-17   1.05500456e-18   3.61171058e-11   4.74089698e-04
    6.26768354e-23   4.56736387e-07]
 [  3.43058394e-12   3.08285662e-44   1.79208104e-23   1.50636124e-18
    9.30133506e-26   2.13561987e-18   1.33183841e-22   5.11428753e-15
    4.46813043e-32   1.00000000e+00]
 [  3.23968655e-14   2.14118405e-42   7.65994724e-26   1.00000000e+00
    3.66010911e-30   1.

Estimator(params=None)

In [27]:
# Configure the accuracy metric for evaluation
metrics = {
    "accuracy":
        learn.MetricSpec(
            metric_fn=tf.metrics.accuracy, prediction_key="classes"),
}

# Evaluate the model and print results
eval_results = mnist_classifier.evaluate(
    x=eval_data, y=eval_labels, metrics=metrics)
print(eval_results)

Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Example conversion:
  est = Estimator(...) -> est = SKCompat(Estimator(...))
Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Example conversion:
  est = Estimator(...) -> est = SKCompat(Estimator(...))


D:\Program Files\Anaconda3\envs\tensorflow_tuts\lib\site-packages\tensorflow\python\util\deprecation.py:247: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.
  equality = a == b


INFO:tensorflow:Starting evaluation at 2017-03-18-18:19:17
INFO:tensorflow:Finished evaluation at 2017-03-18-18:19:18
INFO:tensorflow:Saving dict for global step 2000: accuracy = 0.9785, global_step = 2000, loss = 0.0665391
{'global_step': 2000, 'loss': 0.066539124, 'accuracy': 0.97850001}


{'params': None}